In [17]:
import urllib.request
import pandas as pd

# Load the TSV file
tsv_file_path = './update_tle.tsv'

try:
    sheet_values = pd.read_csv(tsv_file_path, sep='\t')
except FileNotFoundError:
    sheet_values = pd.DataFrame(columns=['tle_0', 'tle_1', 'norad_cat_num'])

# Convert to dictionary for quick lookup
tle_dict = {row['norad_cat_num']: row for _, row in sheet_values.iterrows()}

# Fetch satellite catalog
satcat_url = "https://celestrak.com/pub/satcat.txt" # https://api.optimize.esa.int/data/debris_clouds/satcat.txt
try:
    sat_catalogue = urllib.request.urlopen(satcat_url).read().decode('utf-8').strip().split("\n")
except Exception as e:
    raise ConnectionError(f"Failed to fetch satellite catalog: {e}")

sat_dict = {}
for x in sat_catalogue:
    try:
        sat_dict[x.split()[1]] = x
    except:
        continue


In [26]:
norad_cat_num = '00008'
satellite_data = sat_dict.get(norad_cat_num, None)

print(satellite_data)

1958-004B    00008  *D SPUTNIK 3                 CIS    1958-05-15  TYMSC  1960-04-06     88.4   65.1     255     139   11.8400     


In [18]:
import os
from dotenv import load_dotenv
from spacetrack import SpaceTrackClient

load_dotenv('.env') 

username = os.getenv('SPACETRACK_USERNAME')
password = os.getenv('SPACETRACK_PASSWORD')

st = SpaceTrackClient(username, password)
print(st)


SpaceTrackClient<identity='hello@giacomonanni.info'>


In [28]:
import time
new_entries = []

for key in sat_dict.keys():
    if str('n_' + key) not in tle_dict:
        try:
            if "-" in sat_dict[key][75:85] and "N/A" not in sat_dict[key][119:127]:
                lines = st.tle_latest(norad_cat_id=[key], format='tle')
                lines = lines.split("\n")

                for line1, line2 in zip(*[iter(lines[-3:-1])]*2):
                    update_line = [str(line1), str(line2), str('n_' + key)]
                    print(update_line)
                    new_entries.append(update_line)

                    # Limit the API request
                    time.sleep(2) # 20 advised
            else:
                continue
        except Exception as e:
            print(f"Error processing satellite {key}: {e}")


1957-12-01  20.4200
1958-01-03    N/A  
1958-04-14   0.0800
1970-03-31    N/A  
1958-06-28    N/A  
1958-12-03    N/A  
1959-10-23    N/A  
1959-01-21    N/A  
1959-03-03    N/A  
1959-04-26    N/A  
1961-06-30    N/A  
1959-09-28    N/A  
1959-10-20    N/A  
1960-04-20    N/A  
1959-11-26    N/A  
1960-03-08    N/A  
1961-02-11    N/A  
1961-08-18    N/A  
1967-10-05    N/A  
1960-04-26    N/A  
1960-07-17    N/A  
1962-09-05    N/A  
1960-07-17    N/A  
1965-10-15    N/A  
1961-06-30    N/A  
1960-08-20    N/A  
1960-09-24    N/A  
1960-09-24    N/A  
1960-09-30    N/A  
1960-10-01    N/A  
1974-02-07    N/A  
1960-12-05    N/A  
1960-11-14    N/A  
1968-05-24    N/A  
1960-09-16    N/A  
1960-08-20    N/A  
1960-09-23    N/A  
1960-10-18    N/A  
1960-12-29    N/A  
1960-12-02    N/A  
1960-12-02    N/A  
1961-04-02    N/A  
1961-01-23    N/A  
1970-02-16    N/A  
1973-10-21    N/A  
1961-02-26    N/A  
1961-02-13    N/A  
1961-03-17    N/A  
1961-02-18    N/A  
1961-02-25    N/A  


In [20]:
if new_entries:
    new_df = pd.DataFrame(new_entries, columns=['tle_0', 'tle_1', 'norad_cat_num'])
    updated_sheet = pd.concat([sheet_values, new_df])
    updated_sheet.to_csv(tsv_file_path, sep='\t', index=False)
else:
    print("No new entries to update.")